# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 3:02PM,240884,10,0085817270,ISDIN Corporation,Released
1,Jun 30 2022 3:02PM,240884,10,0085817272,ISDIN Corporation,Released
2,Jun 30 2022 3:02PM,240884,10,0085817273,ISDIN Corporation,Released
3,Jun 30 2022 3:02PM,240884,10,0085817274,ISDIN Corporation,Released
4,Jun 30 2022 3:02PM,240884,10,0085817271,ISDIN Corporation,Released
5,Jun 30 2022 3:02PM,240884,10,0085817275,ISDIN Corporation,Released
6,Jun 30 2022 3:02PM,240884,10,0085817276,ISDIN Corporation,Released
7,Jun 30 2022 3:02PM,240884,10,0085817278,ISDIN Corporation,Released
8,Jun 30 2022 3:02PM,240884,10,0085817279,ISDIN Corporation,Released
9,Jun 30 2022 3:02PM,240884,10,0085817277,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,240881,Released,5
31,240882,Released,1
32,240883,Released,1
33,240884,Released,40
34,240887,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240881,NaN,NaN,5.0
240882,NaN,NaN,1.0
240883,NaN,NaN,1.0
240884,NaN,NaN,40.0
240887,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240781,0.0,0.0,1.0
240784,0.0,0.0,1.0
240786,0.0,0.0,1.0
240800,24.0,29.0,1.0
240804,0.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240781,0,0,1
240784,0,0,1
240786,0,0,1
240800,24,29,1
240804,0,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240781,0,0,1
1,240784,0,0,1
2,240786,0,0,1
3,240800,24,29,1
4,240804,0,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240781,,,1
1,240784,,,1
2,240786,,,1
3,240800,24,29,1
4,240804,,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation
40,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix
41,Jun 30 2022 2:54PM,240883,10,"Methapharm, Inc."
42,Jun 30 2022 2:51PM,240882,19,ACG North America LLC
43,Jun 30 2022 2:49PM,240881,10,"Methapharm, Inc."
48,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc"
49,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation
63,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances
64,Jun 30 2022 2:22PM,240876,12,Hush Hush
71,Jun 30 2022 2:17PM,240875,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,,,40
1,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,,,1
2,Jun 30 2022 2:54PM,240883,10,"Methapharm, Inc.",,,1
3,Jun 30 2022 2:51PM,240882,19,ACG North America LLC,,,1
4,Jun 30 2022 2:49PM,240881,10,"Methapharm, Inc.",,,5
5,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",,,1
6,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,,,14
7,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,,,1
8,Jun 30 2022 2:22PM,240876,12,Hush Hush,,,7
9,Jun 30 2022 2:17PM,240875,10,Emerginnova,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,40,,
1,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,1,,
2,Jun 30 2022 2:54PM,240883,10,"Methapharm, Inc.",1,,
3,Jun 30 2022 2:51PM,240882,19,ACG North America LLC,1,,
4,Jun 30 2022 2:49PM,240881,10,"Methapharm, Inc.",5,,
5,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",1,,
6,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,14,,
7,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,1,,
8,Jun 30 2022 2:22PM,240876,12,Hush Hush,7,,
9,Jun 30 2022 2:17PM,240875,10,Emerginnova,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,40,,
1,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,1,,
2,Jun 30 2022 2:54PM,240883,10,"Methapharm, Inc.",1,,
3,Jun 30 2022 2:51PM,240882,19,ACG North America LLC,1,,
4,Jun 30 2022 2:49PM,240881,10,"Methapharm, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,40.0,NaN,NaN
1,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Jun 30 2022 2:54PM,240883,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Jun 30 2022 2:51PM,240882,19,ACG North America LLC,1.0,NaN,NaN
4,Jun 30 2022 2:49PM,240881,10,"Methapharm, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,40.0,0.0,0.0
1,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Jun 30 2022 2:54PM,240883,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Jun 30 2022 2:51PM,240882,19,ACG North America LLC,1.0,0.0,0.0
4,Jun 30 2022 2:49PM,240881,10,"Methapharm, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2167875,68.0,0.0,0.0
12,481756,8.0,0.0,0.0
15,240847,0.0,29.0,0.0
16,963357,3.0,1.0,0.0
19,1204189,5.0,31.0,24.0
20,1204240,24.0,17.0,0.0
21,722351,3.0,0.0,0.0
22,240846,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2167875,68.0,0.0,0.0
1,12,481756,8.0,0.0,0.0
2,15,240847,0.0,29.0,0.0
3,16,963357,3.0,1.0,0.0
4,19,1204189,5.0,31.0,24.0
5,20,1204240,24.0,17.0,0.0
6,21,722351,3.0,0.0,0.0
7,22,240846,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,68.0,0.0,0.0
1,12,8.0,0.0,0.0
2,15,0.0,29.0,0.0
3,16,3.0,1.0,0.0
4,19,5.0,31.0,24.0
5,20,24.0,17.0,0.0
6,21,3.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,68.0
1,12,Released,8.0
2,15,Released,0.0
3,16,Released,3.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
Executing,0.0,0.0,29.0,1.0,31.0,17.0,0.0,0.0
Released,68.0,8.0,0.0,3.0,5.0,24.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,Executing,0.0,0.0,29.0,1.0,31.0,17.0,0.0,0.0
2,Released,68.0,8.0,0.0,3.0,5.0,24.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,Executing,0.0,0.0,29.0,1.0,31.0,17.0,0.0,0.0
2,Released,68.0,8.0,0.0,3.0,5.0,24.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()